# 📊 Analyse des Candidatures 'Cabine Cibli Job'
#
# **Période d'analyse:** Septembre 2025 - Janvier 2026
#
# Ce notebook fournit une analyse complète des candidatures issues du source 'cabine cibli job', incluant:
# - Statistiques globales (CV, candidatures, clients)
# - Top 10 des clients
# - Répartition par statut et source
# - Analyse détaillée par campagne et client
# - Export complet en Excel


## 📦 Étape 1 : Imports des bibliothèques


In [ ]:
import requests
import pandas as pd
import numpy as np
import json
import warnings
import os
from datetime import datetime, timedelta
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

warnings.filterwarnings('ignore')

print("✅ Bibliothèques chargées avec succès!")


## ⚙️ Étape 2 : Configuration
#
# **MODIFIEZ CES PARAMÈTRES SELON VOS BESOINS**


In [ ]:
# Configuration - Modifier ces variables selon vos besoins
SOURCE_FILTER = "cabine cibli job"  # ← MODIFIEZ LA SOURCE ICI
DATE_START = "2025-09-01"           # ← MODIFIEZ LA DATE DE DÉBUT ICI (YYYY-MM-DD)
DATE_END = "2026-01-14"             # ← MODIFIEZ LA DATE DE FIN ICI (YYYY-MM-DD)
TOP_N_CLIENTS = 15                  # ← Nombre de top clients à afficher
TOP_N_CAMPAIGNS = 15                # ← Nombre de top campagnes à afficher

# Configuration API (NE MODIFIEZ PAS SAUF SI NÉCESSAIRE)
API_URL = "https://api.smart-process-rh.com/v1"
API_KEY = "9TTaz70w8biMjvJ9Q5eIHZwVlQRNmjqAqiNzyGjfeI1S4nubpkSAL1h87FoNrlMv"

HEADERS = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# Créer le dossier 'exports' s'il n'existe pas
os.makedirs("exports", exist_ok=True)

# Afficher la configuration
print("\n✅ Configuration chargée")
print(f"   Source filtrée: {SOURCE_FILTER}")
print(f"   Période: {DATE_START} à {DATE_END}")
print(f"   Top clients à afficher: {TOP_N_CLIENTS}")
print(f"   Top campagnes à afficher: {TOP_N_CAMPAIGNS}")


## 📥 Étape 3 : Récupération des données


In [ ]:
print("\n📥 Récupération des données depuis l'API...")

def load_applications():
    """Charger les candidatures depuis les fichiers locaux ou API"""
    try:
        # Essayer de charger depuis les fichiers locaux
        print("   1️⃣ Chargement des candidatures (fichier local)...")
        applications = pd.read_csv('../stats/applications/raw_applications.csv')
        print(f"      ✓ {len(applications)} candidatures chargées")

        print("   2️⃣ Chargement des campagnes (fichier local)...")
        campaigns = pd.read_csv('../stats/campaigns/raw_campaigns.csv')
        print(f"      ✓ {len(campaigns)} campagnes chargées")

        print("   3️⃣ Chargement des clients (fichier local)...")
        clients = pd.read_csv('../stats/client_stats.csv')
        print(f"      ✓ {len(clients)} clients chargés")

        return applications, campaigns, clients
    except Exception as e:
        print(f"   ⚠️ Erreur lors du chargement local: {e}")
        print("   Tentative de récupération via API...")

        try:
            # Récupérer via API
            print("   1️⃣ Récupération des candidatures (API)...")
            response = requests.get(
                f"{API_URL}/applications/all",
                headers=HEADERS,
                timeout=30
            )
            response.raise_for_status()
            applications = pd.DataFrame(response.json())
            print(f"      ✓ {len(applications)} candidatures récupérées")

            print("   2️⃣ Récupération des campagnes (API)...")
            response = requests.get(
                f"{API_URL}/debug/campaigns",
                headers=HEADERS,
                timeout=30
            )
            response.raise_for_status()
            campaigns = pd.DataFrame(response.json())
            print(f"      ✓ {len(campaigns)} campagnes récupérées")

            print("   3️⃣ Récupération des données client (API)...")
            response = requests.get(
                f"{API_URL}/debug/groups",
                headers=HEADERS,
                timeout=30
            )
            response.raise_for_status()
            clients = pd.DataFrame(response.json())
            print(f"      ✓ {len(clients)} clients récupérés")

            return applications, campaigns, clients
        except Exception as api_error:
            print(f"      ❌ Erreur API: {api_error}")
            raise

applications, campaigns, clients = load_applications()
print("✅ Toutes les données ont été récupérées avec succès!")


## 🔍 Étape 4 : Filtrage et enrichissement des données


In [ ]:
print("\n🔍 Filtrage et enrichissement des données...")

# Convertir les dates et supprimer la timezone
applications['created_at'] = pd.to_datetime(
    applications['created_at'],
    errors='coerce',
    utc=True
).dt.tz_localize(None)

# Filtrer par source
df_filtered = applications[applications['source'] == SOURCE_FILTER].copy()
print(f"   Après filtrage source: {len(df_filtered)} candidatures")

# Filtrer par date
date_start_dt = pd.to_datetime(DATE_START)
date_end_dt = pd.to_datetime(DATE_END)
df_filtered = df_filtered[
    (df_filtered['created_at'] >= date_start_dt) & 
    (df_filtered['created_at'] <= date_end_dt)
]
print(f"   Après filtrage date: {len(df_filtered)} candidatures")

# Enrichir avec les données de campagne (si disponible)
if campaigns is not None and len(campaigns) > 0:
    print("   Enrichissement avec données de campagne...")

print("✅ Filtrage et enrichissement terminés!")


## ============================================================
### 📊 RÉSUMÉ DES STATISTIQUES DE LA CABINE CIBLI
## ============================================================


In [ ]:
# Calculer les métriques globales
total_applications = len(df_filtered)
total_unique_applicants = df_filtered['applicant'].nunique()
total_unique_clients = df_filtered['campaign'].nunique()
applications_with_cv = df_filtered['cv'].notna().sum()

# Afficher le rapport
print("\n" + "="*60)
print("📊 RÉSUMÉ DES STATISTIQUES DE LA CABINE CIBLI")
print("="*60)
print(f"\n✓ Nombre total de CV faits: {applications_with_cv}")
print(f"✓ Nombre total de candidatures: {total_applications}")
print(f"✓ Nombre de clients avec candidatures: {total_unique_clients}")
print(f"\nPériode: {DATE_START} à {DATE_END}")
print(f"Source: {SOURCE_FILTER}")


## 📋 TOP 10 CLIENTS (candidatures)


In [ ]:
print(f"\n📋 TOP {TOP_N_CLIENTS} CLIENTS (candidatures):")

if clients is not None and 'cibli_cabine_applications' in clients.columns:
    top_clients_df = clients.nlargest(TOP_N_CLIENTS, 'cibli_cabine_applications')
    for idx, (_, row) in enumerate(top_clients_df.iterrows(), 1):
        print(f"   {idx:2d}. {row['client_identifier']}: {int(row['cibli_cabine_applications'])} candidatures")
else:
    # Fallback: compter par campagne
    campaign_counts = df_filtered['campaign'].value_counts().head(TOP_N_CLIENTS)
    for idx, (campaign, count) in enumerate(campaign_counts.items(), 1):
        print(f"   {idx:2d}. Campagne {campaign}: {count} candidatures")


## 📊 RÉPARTITION PAR STATUT


In [ ]:
print("\n📊 RÉPARTITION PAR STATUT:")
status_dist = df_filtered['status'].value_counts()
for status, count in status_dist.items():
    percentage = (count / total_applications) * 100
    print(f"  - {status}: {count} ({percentage:.1f}%)")


## 🌐 RÉPARTITION PAR SOURCE


In [ ]:
print("\n🌐 RÉPARTITION PAR SOURCE:")
source_dist = df_filtered['source'].value_counts()
for source, count in source_dist.items():
    percentage = (count / total_applications) * 100
    print(f"  - {source}: {count} ({percentage:.1f}%)")


## 🎯 ANALYSE PAR CAMPAGNE


In [ ]:
print(f"\n🎯 TOP {TOP_N_CAMPAIGNS} CAMPAGNES (candidatures):")

campaign_analysis = []
for campaign, group in df_filtered.groupby('campaign'):
    status_dist_campaign = group['status'].value_counts().to_dict()
    campaign_analysis.append({
        'campaign': campaign,
        'count': len(group),
        'status_distribution': status_dist_campaign,
        'percentage': (len(group) / total_applications) * 100
    })

# Trier par count
campaign_analysis = sorted(campaign_analysis, key=lambda x: x['count'], reverse=True)

for idx, campaign_info in enumerate(campaign_analysis[:TOP_N_CAMPAIGNS], 1):
    print(f"   {idx:2d}. Campagne {campaign_info['campaign']}: {campaign_info['count']} candidatures ({campaign_info['percentage']:.1f}%)")

    # Afficher les 3 top statuts pour cette campagne
    status_dist_campaign = campaign_info['status_distribution']
    for status, count in sorted(status_dist_campaign.items(), key=lambda x: x[1], reverse=True)[:3]:
        pct = (count / campaign_info['count']) * 100
        print(f"      - {status}: {count} ({pct:.1f}%)")


## 🏢 ANALYSE PAR CLIENT (DÉTAILLÉ)


In [ ]:
print(f"\n🏢 ANALYSE DÉTAILLÉE PAR CLIENT:")

if clients is not None and 'cibli_cabine_applications' in clients.columns:
    top_clients_detailed = clients.nlargest(TOP_N_CLIENTS, 'cibli_cabine_applications')

    for idx, (_, client_row) in enumerate(top_clients_detailed.iterrows(), 1):
        client_name = client_row['client_identifier']
        client_applications = client_row['cibli_cabine_applications']

        print(f"\n   {idx:2d}. {client_name}")
        print(f"      Total: {int(client_applications)} candidatures")


## 📈 ANALYSE TEMPORELLE


In [ ]:
print(f"\n📈 RÉPARTITION PAR MOIS:")
df_filtered['month'] = df_filtered['created_at'].dt.to_period('M')
monthly_analysis = df_filtered.groupby('month').agg({
    'id': 'count',
    'applicant': 'nunique',
    'cv': lambda x: x.notna().sum()
}).rename(columns={'id': 'total_applications', 'applicant': 'unique_applicants', 'cv': 'with_cv'})

for month, row in monthly_analysis.iterrows():
    print(f"  {month}: {row['total_applications']:.0f} candidatures, {row['unique_applicants']:.0f} candidats uniques, {row['with_cv']:.0f} avec CV")


## 💾 EXPORT EXCEL


In [ ]:
print("\n💾 Génération du fichier Excel...")

def create_excel_report():
    """Créer un fichier Excel avec tous les résultats"""

    # Créer le workbook
    wb = Workbook()
    wb.remove(wb.active)

    # Styles
    header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=11)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    title_font = Font(bold=True, size=14, color="4472C4")

    # ===== FEUILLE 1: RÉSUMÉ =====
    ws = wb.create_sheet("📋 Résumé", 0)

    ws['A1'] = "📊 ANALYSE CANDIDATURES CABINE CIBLI JOB"
    ws['A1'].font = title_font
    ws.merge_cells('A1:B1')

    ws['A2'] = f"Période: {DATE_START} à {DATE_END}"
    ws['A2'].font = Font(italic=True, size=10)
    ws.merge_cells('A2:B2')

    row = 4
    ws[f'A{row}'] = "Métrique"
    ws[f'B{row}'] = "Valeur"
    ws[f'A{row}'].fill = header_fill
    ws[f'B{row}'].fill = header_fill
    ws[f'A{row}'].font = header_font
    ws[f'B{row}'].font = header_font

    row += 1
    summary_metrics = [
        ("Nombre total de CV faits", applications_with_cv),
        ("Nombre total de candidatures", total_applications),
        ("Nombre unique de candidats", total_unique_applicants),
        ("Nombre de campagnes", total_unique_clients),
        ("Source filtrée", SOURCE_FILTER),
        ("Date de début", DATE_START),
        ("Date de fin", DATE_END),
    ]

    for metric_name, metric_value in summary_metrics:
        ws[f'A{row}'] = metric_name
        ws[f'B{row}'] = metric_value
        ws[f'A{row}'].border = border
        ws[f'B{row}'].border = border
        row += 1

    # ===== FEUILLE 2: TOP CLIENTS =====
    ws = wb.create_sheet("🏢 Top Clients", 1)

    ws['A1'] = "Rang"
    ws['B1'] = "Client"
    ws['C1'] = "Nombre de candidatures"
    for col in ['A', 'B', 'C']:
        ws[f'{col}1'].fill = header_fill
        ws[f'{col}1'].font = header_font

    row = 2
    if clients is not None and 'cibli_cabine_applications' in clients.columns:
        top_clients_df = clients.nlargest(TOP_N_CLIENTS, 'cibli_cabine_applications')
        for idx, (_, client_row) in enumerate(top_clients_df.iterrows(), 1):
            ws[f'A{row}'] = idx
            ws[f'B{row}'] = client_row['client_identifier']
            ws[f'C{row}'] = int(client_row['cibli_cabine_applications'])
            for col in ['A', 'B', 'C']:
                ws[f'{col}{row}'].border = border
            row += 1

    # ===== FEUILLE 3: TOP CAMPAGNES =====
    ws = wb.create_sheet("🎯 Top Campagnes", 2)

    ws['A1'] = "Rang"
    ws['B1'] = "Campagne"
    ws['C1'] = "Nombre de candidatures"
    ws['D1'] = "Pourcentage"
    for col in ['A', 'B', 'C', 'D']:
        ws[f'{col}1'].fill = header_fill
        ws[f'{col}1'].font = header_font

    row = 2
    for idx, campaign_info in enumerate(campaign_analysis[:TOP_N_CAMPAIGNS], 1):
        ws[f'A{row}'] = idx
        ws[f'B{row}'] = str(campaign_info['campaign'])
        ws[f'C{row}'] = campaign_info['count']
        ws[f'D{row}'] = f"{campaign_info['percentage']:.1f}%"
        for col in ['A', 'B', 'C', 'D']:
            ws[f'{col}{row}'].border = border
        row += 1

    # ===== FEUILLE 4: STATUTS =====
    ws = wb.create_sheet("📊 Statuts", 3)

    ws['A1'] = "Statut"
    ws['B1'] = "Nombre"
    ws['C1'] = "Pourcentage"
    for col in ['A', 'B', 'C']:
        ws[f'{col}1'].fill = header_fill
        ws[f'{col}1'].font = header_font

    row = 2
    for status, count in status_dist.items():
        ws[f'A{row}'] = status
        ws[f'B{row}'] = count
        ws[f'C{row}'] = f"{(count/total_applications)*100:.1f}%"
        for col in ['A', 'B', 'C']:
            ws[f'{col}{row}'].border = border
        row += 1

    # ===== FEUILLE 5: SOURCES =====
    ws = wb.create_sheet("🌐 Sources", 4)

    ws['A1'] = "Source"
    ws['B1'] = "Nombre"
    ws['C1'] = "Pourcentage"
    for col in ['A', 'B', 'C']:
        ws[f'{col}1'].fill = header_fill
        ws[f'{col}1'].font = header_font

    row = 2
    for source, count in source_dist.items():
        ws[f'A{row}'] = source
        ws[f'B{row}'] = count
        ws[f'C{row}'] = f"{(count/total_applications)*100:.1f}%"
        for col in ['A', 'B', 'C']:
            ws[f'{col}{row}'].border = border
        row += 1

    # ===== FEUILLE 6: DÉTAIL COMPLET =====
    ws = wb.create_sheet("📊 Détail", 5)

    columns = ['id', 'status', 'source', 'campaign', 'applicant', 'created_at']
    for col_idx, col_name in enumerate(columns, 1):
        cell = ws.cell(row=1, column=col_idx)
        cell.value = col_name
        cell.fill = header_fill
        cell.font = header_font

    # Ajouter les données
    for row_idx, (_, row_data) in enumerate(df_filtered.iterrows(), 2):
        for col_idx, col_name in enumerate(columns, 1):
            cell = ws.cell(row=row_idx, column=col_idx)
            cell.value = str(row_data[col_name])
            cell.border = border

    # Ajuster les largeurs de colonnes
    for ws_name in wb.sheetnames:
        ws = wb[ws_name]
        ws.column_dimensions['A'].width = 25
        ws.column_dimensions['B'].width = 25
        ws.column_dimensions['C'].width = 20
        if ws_name not in ["📋 Résumé"]:
            ws.column_dimensions['D'].width = 15

    # Sauvegarder
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    source_clean = SOURCE_FILTER.replace(" ", "_").replace("/", "_")
    filename = f"../exports/cabine_cibli_analytics_{source_clean}_{timestamp}.xlsx"
    wb.save(filename)

    return filename

excel_file = create_excel_report()
print(f"✅ Fichier Excel généré avec succès!")
print(f"   Fichier: {excel_file}")


## ============================================================
### ✅ ANALYSE TERMINÉE
## ============================================================
#
# Le rapport Excel complet a été généré avec:
# - 📋 Résumé: Métriques clés
# - 🏢 Top Clients: Classement des clients
# - 🎯 Top Campagnes: Candidatures par campagne
# - 📊 Statuts: Distribution des statuts
# - 🌐 Sources: Distribution des sources
# - 📊 Détails: Liste complète des candidatures
